In [1]:
! pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [17]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS

## 1a. Indexing(Document Loading)

In [15]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled

video_id = "DcNRcQkjgm8"

try:
    fetched_transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    transcript_list = fetched_transcript.to_raw_data()
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)
    
except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"An error occurred: {e}")

That is unfriaking believable. That made me cry. This is Jesse Hitler and he's got the craziest resume you've ever seen. He sold a private jet company to Warren Buffett. He helped build Ziko Coconut Water and he's the only guy who can say that 50 Cent was his intern. But I don't admire Jesse just because he's rich financially. I admire him because he leads such a rich life. He runs these crazy endurance races. He goes on adventures with his friends. And I've always wondered, how does one guy do so much? As Jesse says, we don't lack time, we lack a system. So that's why I'm here at his house today to learn the exact system that he uses to plan his year. All right, we're here to do a annual planning session with the one and only Jesse. They always tell you to think like a billionaire. Well, I came to a billionaire's house to see how he thinks about planning his year. >> There he is. >> Welcome to the house, man. What's going on, man? >> Last year, you came on the podcast and you were lik

## 1b. Indexing(Text Splitting)

In [33]:
splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=200)
chunks=splitter.create_documents([transcript])

In [39]:
chunks[0].page_content

"That is unfriaking believable. That made me cry. This is Jesse Hitler and he's got the craziest resume you've ever seen. He sold a private jet company to Warren Buffett. He helped build Ziko Coconut Water and he's the only guy who can say that 50 Cent was his intern. But I don't admire Jesse just because he's rich financially. I admire him because he leads such a rich life. He runs these crazy endurance races. He goes on adventures with his friends. And I've always wondered, how does one guy do so much? As Jesse says, we don't lack time, we lack a system. So that's why I'm here at his house today to learn the exact system that he uses to plan his year. All right, we're here to do a annual planning session with the one and only Jesse. They"

## 1c & 1d (Embedding Generation and Storing in Vector Store)

In [45]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store= FAISS.from_documents(documents=chunks, embedding=embeddings)

In [47]:
vector_store.index_to_docstore_id

{0: 'de11500e-fa85-4c3b-a6c8-f65781b5adb0',
 1: '2b4bb2b3-b881-4060-9fb3-d934df51fea7',
 2: '95a10640-ebe3-4448-bfd6-c79aede18df2',
 3: '2604129b-6008-480c-803b-156e0883121c',
 4: '40adb569-59e8-4c48-b919-1c08c51506d2',
 5: '46732a9b-a19e-4e52-8840-a084e2b99330',
 6: '487f41d9-5f27-4087-b449-bdbfb8fd1c20',
 7: 'cc9e7dd3-b7fb-40d0-90a6-eff536e30a12',
 8: '37db56bd-d226-47d9-b2e4-a9d474cee33e',
 9: '6e6b7840-43f5-402d-81ea-290c69abf6c2',
 10: '73c2b6d2-ad59-412a-9a0c-f98669c02c56',
 11: '9fbaeaae-5c01-49eb-9c95-46f3d5eeab54',
 12: '16e11f01-bbd7-4cbc-83db-3267590bdbb7',
 13: '169eabc2-4317-4345-8cec-be838df9bef7',
 14: '9b7f3bec-bf11-45b5-aab0-8c948df76c50',
 15: '84d0c5c1-bce2-47dd-90bd-79d1e1b7206a',
 16: '8072bba7-cda1-4016-be90-7ebb7a911835',
 17: '6bf26284-3515-432f-9eee-04e26c3eca92',
 18: '01c83722-cce0-4a31-92f2-c4795a60fbd1',
 19: 'de9d791d-d13d-4517-85ee-4bada3b1a78a',
 20: '58cfc3c3-3fb8-4132-8dbd-36dc6fd53a49',
 21: '1220fe90-8309-459f-bbf9-826855af3796',
 22: '41601238-89db-

In [48]:
vector_store.get_by_ids(['de11500e-fa85-4c3b-a6c8-f65781b5adb0'])

[Document(id='de11500e-fa85-4c3b-a6c8-f65781b5adb0', metadata={}, page_content="That is unfriaking believable. That made me cry. This is Jesse Hitler and he's got the craziest resume you've ever seen. He sold a private jet company to Warren Buffett. He helped build Ziko Coconut Water and he's the only guy who can say that 50 Cent was his intern. But I don't admire Jesse just because he's rich financially. I admire him because he leads such a rich life. He runs these crazy endurance races. He goes on adventures with his friends. And I've always wondered, how does one guy do so much? As Jesse says, we don't lack time, we lack a system. So that's why I'm here at his house today to learn the exact system that he uses to plan his year. All right, we're here to do a annual planning session with the one and only Jesse. They")]

In [49]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [50]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1585356d0>, search_kwargs={'k': 2})

In [60]:
question="Is Warren Buffet mentioned in this video? If yes, what is said about him?"

In [61]:
retriever_doc = retriever.invoke(question)

In [55]:
llm = ChatOpenAI(model='gpt-5-nano', temperature=0.2)

In [56]:
prompt = PromptTemplate(
    template="""You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables=["context", "question"]

)

In [57]:
retriever_doc

[Document(id='de11500e-fa85-4c3b-a6c8-f65781b5adb0', metadata={}, page_content="That is unfriaking believable. That made me cry. This is Jesse Hitler and he's got the craziest resume you've ever seen. He sold a private jet company to Warren Buffett. He helped build Ziko Coconut Water and he's the only guy who can say that 50 Cent was his intern. But I don't admire Jesse just because he's rich financially. I admire him because he leads such a rich life. He runs these crazy endurance races. He goes on adventures with his friends. And I've always wondered, how does one guy do so much? As Jesse says, we don't lack time, we lack a system. So that's why I'm here at his house today to learn the exact system that he uses to plan his year. All right, we're here to do a annual planning session with the one and only Jesse. They"),
 Document(id='16e11f01-bbd7-4cbc-83db-3267590bdbb7', metadata={}, page_content="really impactful, really ambitious, really challenging. You know, he'll be the first wor

In [58]:
context_text = "\n\n".join(doc.page_content for doc in retriever_doc)

In [59]:
context_text

"That is unfriaking believable. That made me cry. This is Jesse Hitler and he's got the craziest resume you've ever seen. He sold a private jet company to Warren Buffett. He helped build Ziko Coconut Water and he's the only guy who can say that 50 Cent was his intern. But I don't admire Jesse just because he's rich financially. I admire him because he leads such a rich life. He runs these crazy endurance races. He goes on adventures with his friends. And I've always wondered, how does one guy do so much? As Jesse says, we don't lack time, we lack a system. So that's why I'm here at his house today to learn the exact system that he uses to plan his year. All right, we're here to do a annual planning session with the one and only Jesse. They\n\nreally impactful, really ambitious, really challenging. You know, he'll be the first world's first trillionaire. And that's amazing. I love Elon. I love that he exists, but I don't want to be Elon. And so, it would be foolish for me to choose that

In [62]:
final_prompt = prompt.invoke({"context" : context_text,
                              "question" : question})

In [64]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes. It says that Jesse sold a private jet company to Warren Buffett.


In [68]:
from langchain_core.runnables import RunnableParallel, RunnableSequence, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [66]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [69]:
parallel_chain = RunnableParallel(
    {'context': retriever | RunnableLambda(format_docs),
        'question': RunnablePassthrough()}
)

In [72]:
parser =  StrOutputParser()

In [73]:
final_chain = parallel_chain | prompt | llm | parser

In [74]:
final_chain.invoke("Is Warren Buffet mentioned in this video? If yes, what is said about him?")

'Yes. It says he sold a private jet company to Warren Buffett.'